# Today you are Machine Learning Engineer@ Epic Software Systems, and your task is to develop a new Recommendation System for Opthalmologists (eye doctors)!

### Diabetic Retinopathy (DR) is a pathology that impacts patients with Diabetes Mellitus 2, such that vision is compromised. Base paper on Automated classification of diabetic retinopathy:
https://ieeexplore.ieee.org/abstract/document/6680633
### Diabetic Retinopathy classification using modified AlexNet paper: https://www.sciencedirect.com/science/article/abs/pii/S0045790618334190
### Paper available at https://drive.google.com/file/d/1nl5tYA2jJ1Up_malA8uQmvwHu6UH0k-r/view?usp=sharing


### Automated classification of DR severity can help streamline the treatment process such that patients with higher severity are seen by the doctor first followed by the less severe patiets. This problem of DR clasification is significant since retinal images are biometrics (unique for each individual) which makes generalized modeling difficult and since 90% of the diabetic population that get imaged each year are have NO DR.

### This hard problem can be attacked using the tools of Deep Learning. Today we will show you how to use TensorFlow to construct and train a deep learning model on a small amount of DR data. In the clinic we would like to screen patients with different severity levels of the disease: patients that have NO DR (y=0), vs ones with mild DR (y=1), moderate DR (y=2) and severe DR (y=4). However, in this assignment we're going to make it even easier--the model will just do binary classification: normal images (patients without DR) will be labeled y=0, and those with any amount of DR as y=1.

### The purpose of this assignment is to show you the mechanics of using TensorFlow, not to say that what we present is the state-of-the-art in DR classification--far from it. But the ideas we use today are the first steps in an ML pipeline that can get quite-good performance in classifying diabetic retinopathy :)

### If using Colab, mount your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Change your working directory to the one containing this notebook

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Live_session_notebooks/week_5/')  # you'll have to change this to wherever you stored this notebook
pwd = os.getcwd()
print(pwd)

### Load Tensorflow and (optionally) TensorBoard

In [ ]:
%load_ext tensorboard
from tensorflow import keras

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
#Load the libraries
from datetime import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization

# Task 0: Get familiar with the data

In [ ]:
#Read the data and split into train and validation
from sklearn.model_selection import train_test_split
import h5py
f = h5py.File('/content/drive/MyDrive/Datasets/week_5/1194_DR_smallimages.h5', 'r')  # this is wherever you stored the data
images = f['images']
labels = np.array(f['meta'])
print(f'Number of images = {len(labels)}')
#Plot the image and its label
num = 100
plt.imshow(images[num])
print(f"This image has DR = {labels[num]}")
print(f"Maximum pixel value in image = {np.max(images[num])}")

# Task 1: Data Pre-processing: 
1. Exploratory data analysis (Frequency of labels)
2. Normalization of images in [0, 1] range.
3. Generation of Training, Test data sets (66/33% split)

### Exercise: Plot a histogram illustrating the number of images corresponding to each of the 4 DR severity labels

In [ ]:
### START CODE HERE ###

### END CODE HERE ###

### Exercise: Normalize the images' pixel values so they fall in the range [0, 1]

In [ ]:
### START CODE HERE ###
images_new = None
### END CODE HERE ###
plt.imshow(images_new[10])
print(f"Maximum pixel value in an image = {np.max(images_new[10])}")

### Exercise: Binarize the class labels. Set up 2 one-hot-encoded columns. Column 0 will contain a 1 for every row where the label is 0. Column 1 will contain a 1 for every row where the label is 1, 2, or 3.

### You might be wondering why we don't use a single one-hot-encoded column, since our problem is binary classification. This will become clear in Task 2, when we inspect the AlexNet architecture. 


In [ ]:
### START CODE HERE ###
one_hot_labels = None
### END CODE HERE ###

### Exercise: Perform a 2/3 - 1/3 train-test split on the data

In [ ]:
### START CODE HERE ###
def split_data(mat, target, train_ratio):
    # mat is a matrix of features, target is a matrix of the classification targets
    X_train = None
    X_test  = None
    Y_train = None
    Y_test  = None
    return X_train, X_test, Y_train, Y_test

# Call the function you just defined to create the training and test data
X_train, X_test, Y_train, Y_test = None

### END CODE HERE ###
# Visualize the distribution of the binarized labels for both the training and test data
plt.hist(Y_train[:,1])
plt.hist(Y_test[:,1])
print(f'X_train.shape = {X_train.shape}')
print(f'X_test.shape  = {X_test.shape}')

# Task 2: Model Definition, classification and prediction (No regularization)

## AlexNet architecture

In [ ]:
# Define the AlexNet model [This is given]
#1. Model Definition
class AlexNet(Sequential):
   def __init__(self, input_shape, num_classes):
    super().__init__()

    self.add(Conv2D(96, kernel_size=(11,11), strides= 4,
                    padding= 'valid', activation= 'relu',
                    input_shape= input_shape, kernel_initializer= 'he_normal'))
    self.add(BatchNormalization())
    self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                          padding= 'valid', data_format= None))
    
    
    self.add(Conv2D(256, kernel_size=(5,5), strides= 1,
                    padding= 'same', activation= 'relu',
                    kernel_initializer= 'he_normal'))
    self.add(BatchNormalization())
    self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                          padding= 'valid', data_format= None)) 
    

    self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                    padding= 'same', activation= 'relu',
                    kernel_initializer= 'he_normal'))
    self.add(BatchNormalization())
    
    self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                    padding= 'same', activation= 'relu',
                    kernel_initializer= 'he_normal'))
    self.add(BatchNormalization())
    
    self.add(Conv2D(256, kernel_size=(3,3), strides= 1,
                    padding= 'same', activation= 'relu',
                    kernel_initializer= 'he_normal'))
    self.add(BatchNormalization())
    
    self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                          padding= 'valid', data_format= None))
    

    self.add(Flatten())
    
    self.add(Dense(num_classes, activation= 'softmax'))

    self.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.01),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

### Since our problem involves binary classification, set `num_classes` to 2

In [ ]:
num_classes = 2

### Exercise: Instantiate the model and summarize its architecture using the `.summary()` method of the model instance.

In [ ]:
### START CODE HERE ###
model = None
### END CODE HERE ###

Discuss with your classmates your interpretation of this summary, what do these numbers and names represent (it's okay to speculate)


### Exercise: Train the model, using its `.fit()` method. If you want to use TensorBoard to visualize how its properties evolve over the training epochs, you'll have to create a log directory and employ a TensorBoard callback. We include code for creating the callback below, but you'll have to pass it in as a parameter of the `.fit()` method. 
### In training the model, set aside 20% of the data as a validation set. Use a batch size of 20. Set the output verbosity to 1. Train the model for 40 epochs.

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from helper_functions_for_week5 import *

In [ ]:
##### Comment the following two lines if you don't want to use tensorboard
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
#####

### START CODE HERE ####
training_history = None
### END CODE HERE ###
print("Average test loss: ", np.average(training_history.history['loss']))

### Exercise: Display loss and accuracy for both the training and validation data. You should find the provided functions in `helper_functions_for_week5.py` useful. 

In [ ]:
### START CODE HERE ###

### END CODE HERE ###


In [ ]:
# if you used tensorboard callbacks, try launching tensorboard to view the logs:
# If developing locally: `tensorboard --logdir <THIS_DIRECTORY>/logs
# If developing on Colab: `%tensorboard --logdir logs`

### Exercise: Generate predictions with the test data

In [ ]:
### START CODE HERE ###
prediction_values = None
### END CODE HERE ###

### Exercise: Evaluate the predictions against the Number 1 column of `Y_test`

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score,precision_score,recall_score
### START CODE HERE ###
# Import the necessary functions from sklearn.metrics
print(f'Accuracy = ')
print(f'F1 = ')
print(f'Precision = ')
print(f'Recall = ')
# Display the confusion matrix
print('Confusion matrix =')
### END CODE HERE ###

What do you notice about the performance of this model, especially as it relates to the difference between train and validation accuracy?

### Next, visualize the activations.
Source: https://www.kaggle.com/amarjeet007/visualize-cnn-with-keras

### Exercise: Extract the activations for each layer in our AlexNet model

In [ ]:
### START CODE HERE ###
# Use a list comprehension to extract the output of each layer in the model
layer_outputs = None
# Define a new model with the same input shape as the original model 
# and layer_outputs as its outputs
activation_model = None
# Get all the activations by calling the predict() method on the number 10 image 
# in X_train. Make sure to reshape the image so it has a new 0 axis of length 1.
activations = None
### END CODE HERE ###

### Exercise: Display the first 64 activations of the second model layer (layer index 1) in an 8 x 8 grid. Again, the helper functions are your friends.

In [ ]:
### START CODE HERE ###

### END CODE HERE ###

### Exercise: Display the first 64 activations of the 11th model layer (layer index 10) in an 8 x 8 grid

In [ ]:
### START CODE HERE ###

### END CODE HERE ###

# Comment on the activated feature maps for early layers to later layers. What patterns do you observe?

# Task 3: Regularization by Data Augmentation [Instructor Led]

## We can regularize our model (and, in doing so, hopefully improve it) by augmenting our image data. Image augmentation includes transformations such as rotation, translation, reflection, shearing, and color permutation. TensorFlow provides the ImageDataGenerator class to perform image augmentation in memory; by default, your local storage won't save the newly generated images.

In [ ]:
# Create a Data Generator
data_gen_args = dict(None) # add in the desired parameters of an ImageDataGenerator 
image_datagen = ImageDataGenerator(**data_gen_args)

image_datagen.fit(X_train)
BATCH_SIZE = X_train.shape[0]

### Initialize data generator for training/validation and test data sets

In [ ]:
train_generator = image_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE)
test_generator = image_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE)

### Import libraries and set useful parameters

In [ ]:
seed = 0
aug_batch_size = 6

## Step 1: Understanding the data augmentation process
Images are randomly generated with the options provided in the image data generator

In [ ]:
for e in range(5):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in image_datagen.flow(X_train, Y_train, batch_size=aug_batch_size):
        print(x_batch.shape)
        for i in range(0, aug_batch_size):
            plt.subplot(330+1 + i)
            plt.imshow(x_batch[i], cmap=plt.get_cmap('gray'))
        

        plt.show()
        print(y_batch)
        break

## Step 2: Using augmented data to Train the model
### Visualize using tensorboard

### Method 1: Iterate through randomized `(X, y)` batches generated by `train_generator` and fit AlexNet to each one

In [ ]:
model_reg = AlexNet((np.shape(X_train)[1],np.shape(X_train)[2], 3), num_classes)

In [ ]:
#Method 1: Randomized batches
EPOCHS = 40
reg_history_loss = []
reg_history_vloss = []
reg_history_accuracy = []
reg_history_val_accuracy = []
# Start training
for e in range(EPOCHS):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in train_generator:
        reg_hist = (model_reg.fit(x_batch, y_batch,validation_split=0.2, verbose=1))
        batches += 1
        reg_history_loss.append(reg_hist.history['loss'])
        reg_history_vloss.append(reg_hist.history['val_loss'])
        reg_history_accuracy.append(reg_hist.history['accuracy'])
        reg_history_val_accuracy.append(reg_hist.history['val_accuracy'])
        if batches >= len(X_train) / BATCH_SIZE:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break   


#^ If you have issues with your Colab instance running out of memory, you may have to delete some 
#  previously-created variables such as `model` object holding the unregularized version of AlexNet

### Plot the losses and accuracies for both training and validation data

In [ ]:
#Print the loss and accuracies
print_loss_history({'loss': reg_history_loss, 'val_loss': reg_history_vloss})
print_loss_history({'loss': reg_history_loss, 'val_loss': reg_history_vloss}, logscale=True)
print_accuracy_history({'accuracy': reg_history_accuracy, 'val_accuracy': reg_history_val_accuracy})


### Method 2: Pass `train_generator` into the model's `fit()` method directly and abstract away the looping

In [ ]:
#Method 2
model_reg_2 = AlexNet((np.shape(X_train)[1],np.shape(X_train)[2], 3), num_classes)
reg_history = model_reg_2.fit(
    train_generator,
    steps_per_epoch=50,
    verbose=1, 
    epochs=40,
#    callbacks=[tensorboard_callback]
)

print("Average test loss: ", np.average(reg_history.history['loss']))

In [ ]:
print_loss_history(reg_history.history)
print_loss_history(reg_history.history, logscale=True)
print_loss_accuracy(reg_history.history)

Now we can save the whole trained model to disk:

In [ ]:
model_reg_dir = "reg_model.h5"
model_reg.save(model_reg_dir)  # or model_reg_2 if that's the one you trained

In [ ]:
# %tensorboard --logdir logs

# Task 4: Evaluate the Regularized Model and Report Results

### Exercise: Compute the evaluation metrics

In [ ]:
### START CODE HERE ###
print(f'Accuracy = ')
print(f'F1 = ')
print(f'Precision = ')
print(f'Recall = ')
print('Confusion matrix = ')
### END CODE HERE ###

# Visualize activations from early layers after regularization and report results

In [ ]:
### START CODE HERE ###
activations = None
### END CODE HERE ##
display_activation(activations, 8, 8, 1) #For layer 2
display_activation(activations, 8, 8, 11) #For layer 10